# <center>Generating Music Using GAN</center>

This workbook will implement modified code from [this](https://github.com/corynguyen19/midi-lstm-gan) GitHub repo.


In [79]:
# Imports
from __future__ import print_function, division
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import sys
import pickle
from music21 import converter, instrument, note, chord, stream, duration
from music21 import duration as D
from keras.models import Sequential, Model
from keras.models import load_model
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History
from keras.optimizers import Adam
from keras.backend import argmax
from tensorflow import map_fn

## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [2]:
def get_notes(path):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append([str(element.pitch), element.offset, element.duration])
            elif isinstance(element, chord.Chord):
                song_note = '.'.join(str(n) for n in element.normalOrder)
                song.append([song_note, element.offset, element.duration])
        notes.append(song)

    return notes

def get_notes_with_key(path, filter_key, mode):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
#         Only use music of the same key
        key = midi.analyze('key')
        if(mode==0):
            key_string = str(key.tonic.name)
        elif(mode==1):
            key_string = str(key.mode)
        else:
            key_string = str(key.tonic.name + key.mode)
            
        if(key_string==filter_key):
            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    song.append([str(element.pitch), element.offset, element.duration])
                elif isinstance(element, chord.Chord):
                    song_note = '.'.join(str(n) for n in element.normalOrder)
                    song.append([song_note, element.offset, element.duration])
            notes.append(song)

    return notes

# def get_notes_with_key(path, filter_key, mode):
#     """
#         Gets all notes and chords from midi files where the key matches the string input

#         Parameters
#         ----------
#         path : str
#             The path to the file
#         filter_key : str
#             The string to filter the key on
#         mode : int
#             The type of key used where:
#                 0 - key
#                 1 - major/minor
#                 else - key and major/minor
#     """
#     notes = []

#     for file in glob.glob(path + "*.mid"):        
#         song = []
#         midi = converter.parse(file)
        
#         # Only use music of the same key
#         key = midi.analyze('key')
#         if(mode==0):
#             key_string = str(key.tonic.name)
#         elif(mode==1):
#             key_string = str(key.mode)
#         else:
#             key_string = str(key.tonic.name + key.mode)
            
#         if(key_string==filter_key):
#             print("Parsing %s" % file)
#             notes_to_parse = None

#             try: # file has instrument parts
#                 s2 = instrument.partitionByInstrument(midi)
#                 notes_to_parse = s2.parts[0].recurse() 
#             except: # file has notes in a flat structure
#                 notes_to_parse = midi.flat.notes

#             for element in notes_to_parse:
#                 if isinstance(element, note.Note):
#                     song.append(str(element.pitch))
#                 elif isinstance(element, chord.Chord):
#                     song.append('.'.join(str(n) for n in element.normalOrder))

#     return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 3
input_dir_names = ["test", "Pokemon", "LoZ OOT", "Pokemon GSC", "Pokemon Battle", "Undertale", "ABBA"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
# example of each mode: 0 - C, 1 - major, 2 - Cmajor
# notes = get_notes_with_key(input_path, "minor", 1)
notes = get_notes(input_path)

Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - Cinnabar Island (HGSS Version).mid
Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - S.S. Aqua .mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bicycle.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bug Catching Contest.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Burned Tower.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Champion Battle.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Cherrygrove CityMahogany Town.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dance Theatre.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dark Cave.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dragons Den.mid
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Ecruteak CityCianwood City.mid
Parsing ../Pokemon GSC MIDIs\Pokemon Gol

I will now use an algo to determine the key of each song

In [3]:
def print_key(path):
    key_count = dict()
    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        key = midi.analyze('key')
        key_string = key.tonic.name + key.mode
        if (key_string in key_count): 
            key_count[key_string] += 1
        else: 
            key_count[key_string] = 1
        print(key.tonic.name, key.mode)
    return key_count

key_count = print_key(input_path)
# key_count = print_key("../Undertale MIDIs/")
key_count

Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - Cinnabar Island (HGSS Version).mid
G major
Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - S.S. Aqua .mid
G major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
C# major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bicycle.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bug Catching Contest.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Burned Tower.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Champion Battle.mid
G# minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Cherrygrove CityMahogany Town.mid
F major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dance Theatre.mid
A minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dark Cave.mid
A- major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dragons Den.mid
C# minor
Parsing ../Pokemon GSC MIDIs\Pokemon

{'Gmajor': 5,
 'C#major': 1,
 'Eminor': 3,
 'G#minor': 3,
 'Fmajor': 3,
 'Aminor': 3,
 'A-major': 4,
 'C#minor': 3,
 'Cmajor': 8,
 'Emajor': 4,
 'Dmajor': 8,
 'B-minor': 4,
 'E-major': 3,
 'Bmajor': 2,
 'Amajor': 2,
 'Fminor': 1,
 'Cminor': 1,
 'Dminor': 1,
 'F#major': 1,
 'E-minor': 1}

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [4]:
possibleNotes = set([item[0] for sublist in notes for item in sublist])

# Processing for offsets
possibleOffsets = []
possibleDurations = []

# For each song
for index, song in enumerate(notes):
    song_length = len(song)
    
    # For each note, calculate the difference in offset between this and the previous note
    song_offsets = []
    song_durations = []
    for idx in range(song_length):
        offset = offset = round(song[idx][1] - song[idx - 1][1], 3) if idx > 1 else 0.0
        song_offsets.append(offset)
        if offset not in possibleOffsets:
            possibleOffsets.append(offset)
        
        duration = song[idx][2].quarterLength
        song_durations.append(duration)
        if duration not in possibleDurations:
            possibleDurations.append(duration)
            
    # Update the notes to reflect this
    for idx in range(song_length):
        notes[index][idx][1] = song_offsets[idx]
        notes[index][idx][2] = song_durations[idx]

n_notes = len(possibleNotes)
n_offset = len(possibleOffsets)
n_duration = len(possibleDurations)


possibleNotes = np.array(list(possibleNotes))
possibleOffsets = np.array(list(possibleOffsets))
possibleDurations = np.array(list(possibleDurations))
notes = np.array([list([list(subsublist) for subsublist in sublist]) for sublist in notes])
len(possibleNotes), len(possibleOffsets), len(possibleDurations)

(306, 25, 29)

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [5]:
def prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations):
    """ Prepare the sequences used by the Neural Network """
    song_end_indices = []
    sequence_length = 100
    step_size = 1

    # create a dictionary to map pitches to integers
    pitchnames = sorted(possibleNotes)
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    # create a dictionary to map offset to integers
    offsetnames = sorted(possibleOffsets)
    offset_to_int = dict((offset, number) for number, offset in enumerate(offsetnames))
    
    # create a dictionary to map duration to integers
    durationnames = sorted(possibleDurations)
    duration_to_int = dict((duration, number) for number, duration in enumerate(durationnames))
    
    # find number of each possible choice for normalization
    n_notes = len(possibleNotes)
    n_offset = len(possibleOffsets)
    n_duration = len(possibleDurations)

    network_input = []
    network_output_notes = []
    network_output_offset = []
    network_output_duration = []


    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([np.array([note_to_int[row[0]] / float(n_notes), offset_to_int[row[1]] / float(n_offset), duration_to_int[row[2]] / float(n_duration)]) for row in sequence_in])
            network_output_notes.append(np.array([note_to_int[sequence_out[0]]]))
            network_output_offset.append(np.array([offset_to_int[sequence_out[1]]]))
            network_output_duration.append(np.array([duration_to_int[sequence_out[2]]]))
        song_end_indices.append(len(network_input)-1)


    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 3))

    # Make one-hot-encoding
    network_output_notes = np_utils.to_categorical(network_output_notes, num_classes=n_notes)
    network_output_offset = np_utils.to_categorical(network_output_offset, num_classes=n_offset)
    network_output_duration = np_utils.to_categorical(network_output_duration, num_classes=n_duration)


    return (network_input, network_output_notes, network_output_offset, network_output_duration, song_end_indices)

network_input, network_output_notes, network_output_offset, network_output_duration, song_end_indices = prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations)
network_input.shape

(19486, 100, 3)

## Constructing the model


In [96]:
class GAN():
    def __init__(self, rows, network_input, n_notes, n_offset, n_duration):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 3)
        self.latent_dim = 100
        self.disc_loss = []
        self.gen_loss =[]
        self.n_notes = n_notes
        self.n_offset = n_offset
        self.n_duration = n_duration
        self.batch_size = 128
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator(network_input)
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(network_input.shape[1], network_input.shape[2]))
        generated_seq = self.generator(z)

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        self.outputDest = '../output/GAN_' + str(int(time.time())) + '/'
        if not os.path.exists(self.outputDest):
            os.makedirs(self.outputDest)

    def build_discriminator(self, network_input):
        
        input = Input(shape=(network_input.shape[1], network_input.shape[2]))
        lstm_1 = CuDNNLSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True)(input)
        dropout_1 = Dropout(0.3)(lstm_1)
        lstm_2 = Bidirectional(CuDNNLSTM(512, return_sequences=True))(dropout_1)
        dropout_2 = Dropout(0.3)(lstm_2)
        lstm_3 = Bidirectional(CuDNNLSTM(512))(dropout_2)
        dense_1 = Dense(256)(lstm_3)
        dropout_3 = Dropout(0.3)(dense_1)
        output_d = Dense(1, activation='sigmoid')(dropout_3)
        model = Model(inputs=input, outputs=output_d)
        
        model.summary()
        
        return model

#         model = Sequential()
#         model.add(CuDNNLSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
#         model.add(Dropout(0.3))
#         model.add(Bidirectional(CuDNNLSTM(512, return_sequences=True)))
#         model.add(Dropout(0.3))
#         model.add(Bidirectional(CuDNNLSTM(512)))
#         model.add(Dense(256))
#         model.add(Dropout(0.3))
#         model.add(Dense(1, activation='sigmoid'))
        
#         model.summary()

#         seq = Input(shape=self.seq_shape)
#         validity = model(seq)

#         return Model(seq, validity)
      
    def build_generator(self):

#         model = Sequential()
#         model.add(Dense(128, input_dim=self.latent_dim))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(Dense(256))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
#         model.add(Reshape(self.seq_shape))
#         model.summary()
        
#         noise = Input(shape=(self.latent_dim,))
#         seq = model(noise)

#         return Model(noise, seq)
    
        input_g = Input(shape=(self.latent_dim, network_input.shape[2]))
        dense_g_1 = Dense(128, input_dim=(self.latent_dim, network_input.shape[2]))(input_g)
        lrelu_g_1 = LeakyReLU(alpha=0.2)(dense_g_1)
        batch_norm_g_1 = BatchNormalization(momentum=0.8)(lrelu_g_1)
        dense_g_2 = Dense(256)(batch_norm_g_1)
        lrelu_g_2 = LeakyReLU(alpha=0.2)(dense_g_2)
        batch_norm_g_2 = BatchNormalization(momentum=0.8)(lrelu_g_2)
        output_notes = Dense(self.n_notes, activation='tanh')(batch_norm_g_2)
        output_offset = Dense(self.n_offset, activation='tanh')(batch_norm_g_2)
        output_duration = Dense(self.n_duration, activation='tanh')(batch_norm_g_2)
        x = concatenate([output_notes, output_offset, output_duration])
        dense_g_3 = Dense(3, activation='tanh')(x)
        
        model = Model(inputs=input_g, outputs=dense_g_3)
        
        model.summary()
        
        return model

    def train(self, X_train, y_train, epochs, batch_size=128, sample_interval=50):
        
        self.batch_size = batch_size

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs+1):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            # Add some random noise
            noise1 = np.random.normal(0, 1, (batch_size, self.latent_dim))
            noise2 = np.random.normal(0, 1, (batch_size, self.latent_dim))
            noise3 = np.random.normal(0, 1, (batch_size, self.latent_dim))
            noise = []
            for idx in range(batch_size):
                noise.append([noise1[idx], noise2[idx], noise3[idx]])
            noise = np.asarray(noise)
            noise = np.transpose(noise, (0, 2, 1))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise1 = np.random.normal(0, 1, (batch_size, self.latent_dim))
            noise2 = np.random.normal(0, 1, (batch_size, self.latent_dim))
            noise3 = np.random.normal(0, 1, (batch_size, self.latent_dim))
            noise = []
            for idx in range(batch_size):
                noise.append([noise1[idx], noise2[idx], noise3[idx]])
            noise = np.asarray(noise)
            noise = np.transpose(noise, (0, 2, 1))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            self.disc_loss.append(d_loss[0])
            self.gen_loss.append(g_loss)
            
              # Save model often
            if epoch % sample_interval == 0:
                self.generator.save(self.outputDest + "GANmodel_weights_" + str(epoch) + ".hdf5")
                
    def generate_notes(self, input_notes, possibleNotes, possibleOffsets, possibleDurations):
        # Get pitch names and store in a dictionary
        notes = input_notes
        # create a dictionary to map pitches to integers
        pitchnames = sorted(possibleNotes)
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

        # create a dictionary to map offset to integers
        offsetnames = sorted(possibleOffsets)
        int_to_offset = dict((number, offset) for number, offset in enumerate(offsetnames))

        # create a dictionary to map duration to integers
        durationnames = sorted(possibleDurations)
        int_to_duration = dict((number, duration) for number, duration in enumerate(durationnames))

        # find number of each possible choice for normalization
        n_notes = len(possibleNotes)
        n_offset = len(possibleOffsets)
        n_duration = len(possibleDurations)
        
        # Use random noise to generate sequences
        noise1 = np.random.normal(0, 1, (1, self.latent_dim))
        noise2 = np.random.normal(0, 1, (1, self.latent_dim))
        noise3 = np.random.normal(0, 1, (1, self.latent_dim))
        noise = [noise1, noise2, noise3]
        noise = np.asarray(noise)
        noise = np.transpose(noise, (1, 2, 0))
        predictions = self.generator.predict(noise)
        
        pred_notes = [(x[0]+1)*n_notes/2 for x in predictions[0]]
        pred_notes = [int_to_note[min(int(x), n_notes - 1)] for x in pred_notes]
        pred_offset = [(x[1]+1)*n_offset/2 for x in predictions[0]]
        pred_offset = [int_to_offset[min(int(x), n_offset - 1)] for x in pred_offset]
        pred_duration = [(x[2]+1)*n_duration/2 for x in predictions[0]]
        pred_duration = [int_to_duration[min(int(x), n_duration - 1)] for x in pred_duration]
        pred_full = []
        for (note, offset, duration) in zip(pred_notes, pred_offset, pred_duration): 
            pred_full.append([note, offset, duration])
        print(pred_full)
        
        return pred_full
        
    def create_midi(self, prediction_output, filename):
        """ convert the output from the prediction to notes and create a midi file
            from the notes """
        offset = 0
        output_notes = []

        # create note and chord objects based on the values generated by the model
        count = 0
        for pattern in prediction_output:
            note_str = pattern[0]
            offset_str = pattern[1]
            duration_str = pattern[2]
            if "#-" in note_str:# To fix a rare exception using 2 accidentals
                continue
            # pattern is a chord
            if ('.' in note_str) or note_str.isdigit():
                notes_in_chord = note_str.split('.')
                notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                new_note.duration = D.Duration(float(duration_str))
                output_notes.append(new_chord)
            # pattern is a note
            else:
                new_note = note.Note(note_str)
                new_note.offset = offset
                new_note.duration = D.Duration(float(duration_str))
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)
            # increase offset each iteration so that notes do not stack
            offset += (float(prediction_output[count + 1][1])) if (count + 1 < len(prediction_output)) else 0
            count += 1

        midi_stream = stream.Stream(output_notes)
        midi_stream.write('midi', fp='{}.mid'.format(filename))
        
    def generate_midi_from_saves(self, input_notes):
        # Have each model make a song
        count = 0
        for model_path in glob.glob(self.outputDest + "*.hdf5"):
            print("Composing from %s" % model_path)
            self.generator.load_weights(model_path)
            prediction_notes = self.generate_notes(input_notes)
            self.create_midi(prediction_notes, self.outputDest + 'GAN_output_' + str(count))
            print(self.outputDest + 'GAN_output_' + str(count))
            count += 1
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig(self.outputDest + 'GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [97]:
gan = GAN(100, network_input, n_notes, n_offset, n_duration)
gan.train(X_train=network_input, y_train=[network_output_notes, network_output_offset, network_output_duration], epochs=1000, batch_size=20, sample_interval=100)
gan.plot_loss()

Model: "model_90"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_92 (InputLayer)        (None, 100, 3)            0         
_________________________________________________________________
cu_dnnlstm_94 (CuDNNLSTM)    (None, 100, 512)          1058816   
_________________________________________________________________
dropout_94 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
bidirectional_63 (Bidirectio (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_95 (Dropout)         (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_64 (Bidirectio (None, 1024)              6299648   
_________________________________________________________________
dense_251 (Dense)            (None, 256)               262

66 [D loss: 1.430605, acc.: 50.00%] [G loss: 0.249788]
67 [D loss: 1.344798, acc.: 50.00%] [G loss: 0.261683]
68 [D loss: 1.396559, acc.: 50.00%] [G loss: 0.253813]
69 [D loss: 1.405634, acc.: 50.00%] [G loss: 0.254909]
70 [D loss: 1.388078, acc.: 50.00%] [G loss: 0.253174]
71 [D loss: 1.400743, acc.: 50.00%] [G loss: 0.251117]
72 [D loss: 1.362457, acc.: 50.00%] [G loss: 0.246912]
73 [D loss: 1.362908, acc.: 50.00%] [G loss: 0.245994]
74 [D loss: 1.368716, acc.: 50.00%] [G loss: 0.239294]
75 [D loss: 1.388490, acc.: 50.00%] [G loss: 0.237657]
76 [D loss: 1.400863, acc.: 50.00%] [G loss: 0.246789]
77 [D loss: 1.423239, acc.: 50.00%] [G loss: 0.239175]
78 [D loss: 1.368089, acc.: 50.00%] [G loss: 0.240368]
79 [D loss: 1.360234, acc.: 50.00%] [G loss: 0.238058]
80 [D loss: 1.367920, acc.: 50.00%] [G loss: 0.204282]
81 [D loss: 1.409402, acc.: 50.00%] [G loss: 0.248091]
82 [D loss: 1.377644, acc.: 50.00%] [G loss: 0.221387]
83 [D loss: 1.400956, acc.: 50.00%] [G loss: 0.206578]
84 [D loss

213 [D loss: 1.279308, acc.: 50.00%] [G loss: 0.126625]
214 [D loss: 1.233721, acc.: 50.00%] [G loss: 0.156105]
215 [D loss: 1.263687, acc.: 50.00%] [G loss: 0.152446]
216 [D loss: 1.215186, acc.: 50.00%] [G loss: 0.156286]
217 [D loss: 1.223152, acc.: 50.00%] [G loss: 0.151936]
218 [D loss: 1.227067, acc.: 50.00%] [G loss: 0.148874]
219 [D loss: 1.266294, acc.: 50.00%] [G loss: 0.155171]
220 [D loss: 1.232980, acc.: 50.00%] [G loss: 0.148394]
221 [D loss: 1.240356, acc.: 50.00%] [G loss: 0.154852]
222 [D loss: 1.239254, acc.: 50.00%] [G loss: 0.146287]
223 [D loss: 1.242967, acc.: 50.00%] [G loss: 0.162219]
224 [D loss: 1.240829, acc.: 50.00%] [G loss: 0.151360]
225 [D loss: 1.236548, acc.: 50.00%] [G loss: 0.148991]
226 [D loss: 1.226550, acc.: 50.00%] [G loss: 0.157267]
227 [D loss: 1.237800, acc.: 50.00%] [G loss: 0.153954]
228 [D loss: 1.265351, acc.: 50.00%] [G loss: 0.153623]
229 [D loss: 1.249692, acc.: 50.00%] [G loss: 0.143111]
230 [D loss: 1.223758, acc.: 50.00%] [G loss: 0.

360 [D loss: 1.280960, acc.: 50.00%] [G loss: 0.147367]
361 [D loss: 1.270158, acc.: 50.00%] [G loss: 0.156091]
362 [D loss: 1.282907, acc.: 50.00%] [G loss: 0.156145]
363 [D loss: 1.266995, acc.: 50.00%] [G loss: 0.149058]
364 [D loss: 1.294390, acc.: 50.00%] [G loss: 0.144455]
365 [D loss: 1.278597, acc.: 50.00%] [G loss: 0.144148]
366 [D loss: 1.281391, acc.: 50.00%] [G loss: 0.154842]
367 [D loss: 1.266895, acc.: 50.00%] [G loss: 0.141639]
368 [D loss: 1.283054, acc.: 50.00%] [G loss: 0.140416]
369 [D loss: 1.284154, acc.: 50.00%] [G loss: 0.136395]
370 [D loss: 1.252487, acc.: 50.00%] [G loss: 0.142266]
371 [D loss: 1.242180, acc.: 50.00%] [G loss: 0.147006]
372 [D loss: 1.258936, acc.: 50.00%] [G loss: 0.145261]
373 [D loss: 1.272801, acc.: 50.00%] [G loss: 0.150180]
374 [D loss: 1.235965, acc.: 50.00%] [G loss: 0.140100]
375 [D loss: 1.304179, acc.: 50.00%] [G loss: 0.161018]
376 [D loss: 1.231323, acc.: 50.00%] [G loss: 0.145490]
377 [D loss: 1.286110, acc.: 50.00%] [G loss: 0.

507 [D loss: 1.300934, acc.: 50.00%] [G loss: 0.167985]
508 [D loss: 1.285643, acc.: 50.00%] [G loss: 0.139546]
509 [D loss: 1.323777, acc.: 50.00%] [G loss: 0.137391]
510 [D loss: 1.292312, acc.: 50.00%] [G loss: 0.135971]
511 [D loss: 1.298753, acc.: 50.00%] [G loss: 0.153370]
512 [D loss: 1.260618, acc.: 50.00%] [G loss: 0.152341]
513 [D loss: 1.237642, acc.: 50.00%] [G loss: 0.147192]
514 [D loss: 1.281277, acc.: 50.00%] [G loss: 0.147510]
515 [D loss: 1.255839, acc.: 50.00%] [G loss: 0.163665]
516 [D loss: 1.266855, acc.: 50.00%] [G loss: 0.148340]
517 [D loss: 1.279294, acc.: 50.00%] [G loss: 0.157637]
518 [D loss: 1.258950, acc.: 50.00%] [G loss: 0.159106]
519 [D loss: 1.290887, acc.: 50.00%] [G loss: 0.146183]
520 [D loss: 1.304037, acc.: 50.00%] [G loss: 0.144007]
521 [D loss: 1.252163, acc.: 50.00%] [G loss: 0.153089]
522 [D loss: 1.273152, acc.: 50.00%] [G loss: 0.161849]
523 [D loss: 1.265572, acc.: 50.00%] [G loss: 0.153743]
524 [D loss: 1.259629, acc.: 50.00%] [G loss: 0.

654 [D loss: 1.226105, acc.: 50.00%] [G loss: 0.165290]
655 [D loss: 1.235533, acc.: 50.00%] [G loss: 0.156759]
656 [D loss: 1.225272, acc.: 50.00%] [G loss: 0.141032]
657 [D loss: 1.223979, acc.: 50.00%] [G loss: 0.162021]
658 [D loss: 1.276590, acc.: 50.00%] [G loss: 0.143123]
659 [D loss: 1.206403, acc.: 50.00%] [G loss: 0.160376]
660 [D loss: 1.260408, acc.: 50.00%] [G loss: 0.159157]
661 [D loss: 1.258332, acc.: 50.00%] [G loss: 0.164065]
662 [D loss: 1.222783, acc.: 50.00%] [G loss: 0.163157]
663 [D loss: 1.242417, acc.: 50.00%] [G loss: 0.149770]
664 [D loss: 1.268168, acc.: 50.00%] [G loss: 0.151351]
665 [D loss: 1.260299, acc.: 50.00%] [G loss: 0.166296]
666 [D loss: 1.199688, acc.: 50.00%] [G loss: 0.171249]
667 [D loss: 1.254137, acc.: 50.00%] [G loss: 0.151925]
668 [D loss: 1.243309, acc.: 50.00%] [G loss: 0.151182]
669 [D loss: 1.263838, acc.: 50.00%] [G loss: 0.152332]
670 [D loss: 1.246006, acc.: 50.00%] [G loss: 0.156324]
671 [D loss: 1.234976, acc.: 50.00%] [G loss: 0.

801 [D loss: 1.238474, acc.: 50.00%] [G loss: 0.142958]
802 [D loss: 1.224429, acc.: 50.00%] [G loss: 0.150868]
803 [D loss: 1.202495, acc.: 50.00%] [G loss: 0.165452]
804 [D loss: 1.192999, acc.: 50.00%] [G loss: 0.187066]
805 [D loss: 1.259037, acc.: 50.00%] [G loss: 0.152701]
806 [D loss: 1.256736, acc.: 50.00%] [G loss: 0.161970]
807 [D loss: 1.267005, acc.: 50.00%] [G loss: 0.157839]
808 [D loss: 1.239706, acc.: 50.00%] [G loss: 0.169847]
809 [D loss: 1.212598, acc.: 50.00%] [G loss: 0.160477]
810 [D loss: 1.249415, acc.: 50.00%] [G loss: 0.158476]
811 [D loss: 1.253202, acc.: 50.00%] [G loss: 0.153907]
812 [D loss: 1.209390, acc.: 50.00%] [G loss: 0.151904]
813 [D loss: 1.244630, acc.: 50.00%] [G loss: 0.160293]
814 [D loss: 1.238591, acc.: 50.00%] [G loss: 0.166669]
815 [D loss: 1.213434, acc.: 50.00%] [G loss: 0.161050]
816 [D loss: 1.194543, acc.: 50.00%] [G loss: 0.156842]
817 [D loss: 1.198432, acc.: 50.00%] [G loss: 0.153775]
818 [D loss: 1.184572, acc.: 50.00%] [G loss: 0.

948 [D loss: 1.237245, acc.: 50.00%] [G loss: 0.161446]
949 [D loss: 1.231264, acc.: 50.00%] [G loss: 0.161676]
950 [D loss: 1.215908, acc.: 50.00%] [G loss: 0.167367]
951 [D loss: 1.213404, acc.: 50.00%] [G loss: 0.168252]
952 [D loss: 1.230540, acc.: 50.00%] [G loss: 0.154625]
953 [D loss: 1.215276, acc.: 50.00%] [G loss: 0.165913]
954 [D loss: 1.184784, acc.: 50.00%] [G loss: 0.170434]
955 [D loss: 1.229411, acc.: 50.00%] [G loss: 0.166457]
956 [D loss: 1.212630, acc.: 50.00%] [G loss: 0.178673]
957 [D loss: 1.180664, acc.: 50.00%] [G loss: 0.174339]
958 [D loss: 1.233714, acc.: 50.00%] [G loss: 0.161743]
959 [D loss: 1.276895, acc.: 50.00%] [G loss: 0.179430]
960 [D loss: 1.262033, acc.: 50.00%] [G loss: 0.171348]
961 [D loss: 1.244110, acc.: 50.00%] [G loss: 0.194759]
962 [D loss: 1.287337, acc.: 50.00%] [G loss: 0.139517]
963 [D loss: 1.304054, acc.: 50.00%] [G loss: 0.154328]
964 [D loss: 1.246181, acc.: 50.00%] [G loss: 0.158758]
965 [D loss: 1.242712, acc.: 50.00%] [G loss: 0.

In [98]:
output_pred = gan.generate_notes(notes, possibleNotes, possibleOffsets, possibleDurations)

[['B3', 2.5, 8.0], ['G1', 0.0, 3.5], ['D6', 2.25, Fraction(1, 3)], ['5.11', 2.5, 5.75], ['G3', Fraction(333, 1000), 3.0], ['6.11.0', 3.0, 1.5], ['G5', 1.0, 1.75], ['F#6', 1.75, 7.0], ['E5', 3.0, 8.0], ['G4', 2.0, 1.25], ['4.6.7', 3.0, 5.75], ['G6', 2.25, 2.75], ['G3', 0.25, 1.75], ['G1', 2.0, 4.0], ['G#6', 4.5, 8.0], ['G6', 0.0, 1.5], ['G6', 3.0, Fraction(8, 3)], ['G6', 2.0, Fraction(2, 3)], ['G6', 0.5, 3.0], ['G#3', 1.667, 0.75], ['8.9.0', 0.417, 6.0], ['F5', 0.25, 2.0], ['G5', 1.667, 2.25], ['F1', 0.833, 3.0], ['G#6', 0.833, 0.5], ['8.10.1', 0.75, 4.5], ['11.1.6', 3.75, 4.25], ['A5', 1.25, 5.75], ['E-6', 3.0, Fraction(1, 3)], ['4.6.8.11.1', 2.5, Fraction(11, 3)], ['9.10', 2.25, Fraction(2, 3)], ['G4', Fraction(667, 1000), 1.75], ['G4', 0.833, 4.0], ['4.8.11', 2.5, 5.75], ['G2', Fraction(333, 1000), 3.0], ['7.10', 2.5, Fraction(4, 3)], ['G4', 3.0, Fraction(11, 3)], ['G6', 4.0, 0.25], ['D2', 2.0, Fraction(1, 3)], ['E6', 1.667, 3.75], ['G6', 3.75, 4.0], ['B1', 0.0, 3.75], ['6.9.11.2', 2

In [99]:
gan.create_midi(output_pred, 'GAN_output_X')

In [111]:
count = 0
for model_path in glob.glob(gan.outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    gan.generator.load_weights(model_path)
    prediction_notes = gan.generate_notes(possibleNotes)
    gan.create_midi(prediction_notes, gan.outputDest + 'GAN_output_' + str(count))
    print(gan.outputDest + 'GAN_output_' + str(count))
    count += 1

Composing from ../output/GAN_1570437586\GANmodel_weights_0.hdf5
[[[-0.79037344]
  [ 0.70460176]
  [-0.28947598]
  [ 0.8814572 ]
  [-0.6107855 ]
  [-0.38538986]
  [ 0.07783478]
  [ 0.41338134]
  [-0.8948215 ]
  [ 0.7541267 ]
  [ 0.42156288]
  [ 0.82168484]
  [-0.877071  ]
  [-0.3693905 ]
  [ 0.06191757]
  [-0.41833857]
  [ 0.6497538 ]
  [ 0.9913109 ]
  [ 0.9248787 ]
  [-0.99073195]
  [ 0.75921595]
  [-0.38791367]
  [ 0.7617763 ]
  [-0.08390729]
  [ 0.38214526]
  [-0.53922236]
  [ 0.99005866]
  [ 0.86944044]
  [ 0.02686271]
  [-0.9834003 ]
  [-0.44417801]
  [ 0.947354  ]
  [-0.4138857 ]
  [ 0.30625492]
  [ 0.8054296 ]
  [ 0.8645084 ]
  [-0.91852987]
  [-0.82418436]
  [-0.9886027 ]
  [ 0.2925736 ]
  [-0.9160985 ]
  [ 0.85405934]
  [ 0.70144534]
  [ 0.9964825 ]
  [ 0.19647111]
  [ 0.9931008 ]
  [-0.90167433]
  [ 0.8092278 ]
  [ 0.9887396 ]
  [ 0.9437407 ]
  [-0.97892994]
  [-0.27275044]
  [-0.42333934]
  [-0.60200846]
  [ 0.9684587 ]
  [-0.89766115]
  [ 0.79444194]
  [ 0.92340887]
  [-0.90

[[[-0.31863266]
  [-0.04901373]
  [ 0.9647185 ]
  [-0.9970886 ]
  [-0.7901506 ]
  [-0.5245192 ]
  [-0.99598765]
  [-0.98946106]
  [-0.97796994]
  [-0.7766503 ]
  [-0.9589379 ]
  [-0.90780485]
  [-0.89653635]
  [-0.83506596]
  [-0.9946219 ]
  [-0.698596  ]
  [ 0.70834357]
  [ 0.33375612]
  [-0.83358085]
  [-0.9176603 ]
  [-0.88028145]
  [-0.8599078 ]
  [ 0.9162866 ]
  [ 0.13114238]
  [-0.6726755 ]
  [-0.8211164 ]
  [-0.46654317]
  [ 0.22901817]
  [-0.96735495]
  [-0.28233814]
  [ 0.6000631 ]
  [ 0.6072935 ]
  [ 0.7140057 ]
  [ 0.82475835]
  [-0.14915876]
  [-0.96274424]
  [ 0.8794212 ]
  [-0.44175673]
  [ 0.39304227]
  [ 0.7064495 ]
  [-0.8201694 ]
  [ 0.12927328]
  [-0.85720026]
  [ 0.00342063]
  [ 0.10993685]
  [ 0.8549226 ]
  [-0.8549247 ]
  [-0.9479352 ]
  [-0.92161894]
  [-0.8428619 ]
  [ 0.01836698]
  [ 0.88301367]
  [ 0.4289415 ]
  [-0.65191066]
  [ 0.9797393 ]
  [ 0.300754  ]
  [-0.27318373]
  [-0.23616558]
  [ 0.92543495]
  [-0.91959906]
  [ 0.1948324 ]
  [ 0.7325499 ]
  [ 0.99

../output/GAN_1570437586/GAN_output_9
Composing from ../output/GAN_1570437586\GANmodel_weights_90.hdf5
[[[-0.99372005]
  [-0.07109336]
  [ 0.50417745]
  [ 0.6114273 ]
  [-0.99372274]
  [-0.93069106]
  [ 0.14586872]
  [-0.9409094 ]
  [-0.89703465]
  [-0.6764112 ]
  [ 0.91217065]
  [-0.08329976]
  [-0.6026658 ]
  [-0.9155475 ]
  [-0.4573485 ]
  [-0.2710771 ]
  [-0.83602333]
  [-0.9268878 ]
  [ 0.4406684 ]
  [ 0.5563792 ]
  [ 0.9190806 ]
  [-0.56058013]
  [ 0.567446  ]
  [ 0.8114215 ]
  [ 0.8143599 ]
  [-0.43429366]
  [-0.43725628]
  [ 0.1867322 ]
  [-0.52914596]
  [-0.9893395 ]
  [ 0.60228133]
  [-0.9499852 ]
  [-0.8755273 ]
  [ 0.9668063 ]
  [-0.04185651]
  [-0.33947974]
  [ 0.90217185]
  [ 0.85657346]
  [ 0.14158826]
  [-0.539097  ]
  [-0.33338523]
  [-0.90614957]
  [ 0.562485  ]
  [ 0.6537802 ]
  [ 0.3161599 ]
  [ 0.01375579]
  [-0.38288748]
  [ 0.56375265]
  [ 0.90417427]
  [ 0.99968565]
  [ 0.92991376]
  [ 0.87509125]
  [ 0.91637343]
  [ 0.95765615]
  [-0.5329507 ]
  [-0.8507352 ]
 

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [7]:
def generate_notes(model, notes, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    pitchnames = sorted(notes)
    
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
#         prediction_input = (prediction_input / float(n_vocab))*2 - 1

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]
    
    print(prediction_output)

    return prediction_output

# prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)

Next, I will create a midi using these notes and save to a file

In [8]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
# create_midi(prediction_output, outputDest + 'LSTM_output_final')

Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [ ]:
# Have each model make a song
count = 0
for model_path in glob.glob(outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1